In [14]:
from langchain_openai  import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
import json

In [3]:
model = ChatOpenAI(
    base_url="https://ws-02.wade0426.me/v1",
    api_key="day3hw",
    model="Qwen/Qwen3-VL-8B-Instruct",
    max_tokens=50,
    temperature=0
)

In [4]:
@tool
def extract_order_data(name: str, phone: str, product: str, quantity: int, address: str):
    """
    資料提取專用工具
    專門用於從非結構話文本中提取訂單相關資訊（姓名，電話，商品，數量，地址）。
    """

    return {
        "name": name,
        "phone": phone,
        "product": product,
        "quantity": quantity,
        "address": address
    }

In [ ]:
{
    "name": "extract_order_data",
    "description": "資料提取專用工具。專們用於從非結構文本中提取訂單相關資訊...",
    "parameters":{
        "type":"object",
        "properties":{
            "name":{"type":"string"},
            "phone":{"type":"string"},
            "product":{"type":"string"},
            "quantity":{"type":"integer"},
            "address":{"type":"string"},
        },
        "required":["name","phone","product","quantity","address"]
    }
}

In [9]:
llm_with_tools = model.bind_tools([extract_order_data])

system_prompt = ChatPromptTemplate.from_messages({
    ("system", "你是一個精準的訂單管理員，請從對話中提取訂單資訊。"),
    ("user", "{user_input}")
})

def extract_tool_args(ai_message):
    if ai_message.tool_calls:
        return ai_message.tool_calls[0]['args']
    return None

chain = system_prompt | llm_with_tools | extract_tool_args

user_text = input("請輸入：")

result = chain.invoke({"user_input": user_text})

if result:
    print("提取成功: ")
    print(json.dumps(result, ensure_ascii=False,indent=2))
else:
    print("提取失敗")

提取成功: 
{
  "name": "Eric",
  "phone": "0912-123-123",
  "product": "筆記型電腦",
  "quantity": 3,
  "address": "台中是北區, 下週五送達"
}


In [11]:
user_input ="你好，我是Eric, 電話是 0912-123-123, 我想訂購3台筆記型電腦，下週五送到台中是北區。"
ai_msg = llm_with_tools.invoke(user_input)
ai_msg

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 395, 'total_tokens': 466, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gemma-3-27b-it.gguf', 'system_fingerprint': 'b0-unknown', 'id': 'chatcmpl-TzcJPXU9Kvui6pfFN4kfSBuTUwY5INHv', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c269e-0862-7fd2-875e-10c8407d173d-0', tool_calls=[{'name': 'extract_order_data', 'args': {'name': 'Eric', 'phone': '0912-123-123', 'product': '筆記型電腦', 'quantity': 3, 'address': '台中是北區。下週五'}, 'id': 'yUbb03DXkn74iRYozecAKfnjBbdJ3zRK', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 395, 'output_tokens': 71, 'total_tokens': 466, 'input_token_details': {}, 'output_token_details': {}})

In [13]:
def extract_tools_args(ai_message):
    if ai_message.tool_calls:
        return ai_message.tool_calls[0]['args']
    else:
        return ai_message.content

chain = system_prompt | llm_with_tools | extract_tools_args

while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "q"]:
        print("Bye!")
        break

    result = chain.invoke({"user_input": user_input})

    print(json.dumps(result, ensure_ascii=False,indent=2))

{
  "name": "Eric",
  "phone": "0912-123-123",
  "product": "筆記型電腦",
  "quantity": 3,
  "address": "台中是北區, 下週五送達"
}
"您好！請提供訂單資訊，我會協助您整理。"
{
  "name": "請提供訂單內容，以便我提取資訊。",
  "phone": "請提供訂單內容，以便我提取資訊。",
  "product": "請提供訂單內容，以便我提取資訊。",
  "quantity": 1,
  "address": "請提供訂單內容，以便我提取資訊。"
}
"好的，請提供您要處理的對話內容。"
{
  "name": "Eric",
  "phone": "0912-123-123",
  "product": "筆記型電腦",
  "quantity": 3,
  "address": "台中是北區, 下週五送達"
}


KeyboardInterrupt: Interrupted by user

In [17]:
@tool
def generate_tech_summary(article_content: str):
    """
    科技文章專用摘要生成工具。
    【判斷邏輯】：
    1. 只有當輸入內容屬於「科技」、「程式設計」、「AI」、「軟體工程」或「IT 技術」領域時，才使用此工具。
    2. 如果內容是「閒聊」、「食譜」、「天氣」、「日常日記」等非技術內容，請勿使用此工具。

    功能：將輸入的技術文章歸納出 3 個重點。
    """

    prompt_system = ChatPromptTemplate.from_messages({
            ("system", "你是一個精準的訂單管理員，請從對話中提取訂單資訊。"),
            ("user", "{text}")
    })

    chain = prompt_system | model | StrOutputParser()

    result = chain.invoke({"text": article_content})

    return  result

In [18]:
llm1_with_tools = model.bind_tools([generate_tech_summary])

router_prompt = ChatPromptTemplate.from_messages({
    ("user", "{input}")
})

while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "q"]:
        print("Bye!")
        break

    chain = router_prompt | llm1_with_tools
    ai_msg = chain.invoke({"input": user_input})
    if ai_msg.tool_calls:
        print(f"[決策] 判斷為科技文章")
        tool_args = ai_msg.tool_calls[0]['args']

        final_result = generate_tech_summary.invoke(tool_args)

        print(f"[執行結果]: \n{final_result}")

    else:
        print(f"[決策] 判斷為閒聊/非科技文章，直接回覆。")
        print("[AI resp]: ", {ai_msg.content})

[決策] 判斷為科技文章
[執行結果]: 
好的，我明白了。請提供對話內容，我會盡力從中提取訂單資訊。我會專注於識別以下資訊：

* **訂單項目:** 購買的商品或服務
* **數量:** 每個項目的數量
* **價格:** 每個項目的價格，以及總價
* **客戶資訊:** 客戶姓名、聯絡方式、地址等
* **訂單日期:** 訂單建立或下達的日期
* **付款方式:** 使用的付款方式
* **運送資訊:** 運送地址、運送方式、預計送達日期
* **其他特殊要求:** 例如特殊包裝、禮品訊息等

請將對話內容提供給我，我會盡快處理。

[決策] 判斷為閒聊/非科技文章，直接回覆。
[AI resp]:  {'您好，陳大明先生，我了解您要訂購 3 台筆記型電腦，並希望下週五送到台中市北區。由於這不是技術相關的請求，我無法使用科技摘要工具。我會將您的訂購需求記錄下來，並請相關人員與您聯繫確認訂單細節。'}


KeyboardInterrupt: Interrupted by user